<a href="https://colab.research.google.com/github/gabrielborja/python_data_analysis/blob/main/email_analytics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Working with mbox files

Message format <hr>
The basic Internet message format used for email is defined by RFC 5322.
Internet email messages consist of two sections, 'header' and 'body'. These are known as 'content'.
The header is structured into fields such as From, To, CC, Subject, Date, and other information about the email.
The body contains the message, as unstructured text, sometimes containing a signature block at the end. The header is separated from the body by a blank line.
More information can be found [here](https://en.wikipedia.org/wiki/Email#Header_fields)

## Uploading initial packages and data

In [1]:
#Importing preliminary packages
import numpy as np
import pandas as pd

In [4]:
#Remove previous versions of the uploaded excel file
!rm email_list.json

In [5]:
#Uploading file from local drive
from google.colab import files
uploaded = files.upload()

Saving email_list.json to email_list.json


In [ ]:
#Storing json in a Pandas Dataframe
import io
df = pd.read_json(io.BytesIO(uploaded['email_list.json']), encoding='utf-8')

In [7]:
#Checking the dataframe info
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1480 entries, 0 to 1479
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Date     1480 non-null   object
 1   From     1480 non-null   object
 2   Subject  1480 non-null   object
dtypes: object(3)
memory usage: 34.8+ KB


##Initial Data cleaning

###Converting to datetime

In [8]:
#Create a copy of the dataframe to perform data cleaning and manipulation
emails = df.copy()

In [9]:
#Replace Unknown Timezones: i.e. CDT, CST flaged as warnings by read_json (UnknownTimezoneWarning)
emails['Date'] = emails['Date'].replace(to_replace=r'CDT|CST', value='UTC', regex=True)
#df[df['Date'].str.contains('CDT|CST', regex=True)] #==>Check how many rows contain unknown timezone

In [10]:
#Replace invalid timezone stamps that will result in NaT values
emails['Date'] = emails['Date'].replace(to_replace='\(-05\)|\(GMT-05:00\)', value='(ECT)', regex=True)
#emails[emails['Date'].str.contains('\(-05\)|\(GMT-05:00\)', regex=True)] #==> Check how many rows contain invalid timezone

In [12]:
#Parsing strings to datetime object and converting to Oslo time zone
emails = emails.assign(Event = emails['Date'])
emails['Event'] = pd.to_datetime(emails['Date'], errors='coerce', utc=True, format='%Y-%m-%d %H:%M:%S', infer_datetime_format=True).dt.tz_convert('Europe/Oslo')

In [26]:
#Sorting the 'Event' column
emails = emails.sort_values(by=['Event']).reset_index(drop=True)
emails.tail()

,Date,From,Subject,Event
1475,"Mon, 07 Jun 2021 18:16:00 +0000","""Humble Bundle"" <contact@mailer.humblebundle.com>",Get Data Smart with Humble Bundle and Mercury ...,2021-06-07 20:16:00+02:00
1476,"Mon, 7 Jun 2021 13:57:53 -0500 (ECT)",banco@pichincha.com,NOTIFICACION BANCO PICHINCHA,2021-06-07 20:57:53+02:00
1477,"Mon, 07 Jun 2021 14:03:55 -0600","""Comunicados Banco Pichincha"" <comunicados@pic...",Información_de_COSEDE?,2021-06-07 22:03:55+02:00
1478,"Mon, 07 Jun 2021 22:17:08 +0000",Diners Club del Ecuador <gestion@comunicacione...,Accede al reembolso de tus compras,2021-06-08 00:17:08+02:00
1479,"Mon, 7 Jun 2021 23:13:14 +0000 (UTC)",LinkedIn Job Recommendations <jobs-listings@li...,Harnham is looking for: Data Scientist.,2021-06-08 01:13:14+02:00


In [14]:
#Check that the new dataframe info has all valid datetime
emails.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1480 entries, 0 to 1479
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype                      
---  ------   --------------  -----                      
 0   Date     1480 non-null   object                     
 1   From     1480 non-null   object                     
 2   Subject  1480 non-null   object                     
 3   Event    1480 non-null   datetime64[ns, Europe/Oslo]
dtypes: datetime64[ns, Europe/Oslo](1), object(3)
memory usage: 46.4+ KB


In [20]:
#Checking the presence of () in date
par_date = [d for d in emails['Date'] if '(' in d]
par_date[:10]

['Tue, 25 May 2021 17:14:59 +0000 (UTC)',
 'Thu, 03 Jun 2021 05:48:41 +0000 (UTC)',
 'Sun, 6 Jun 2021 11:41:03 -0500 (ECT)',
 'Sun, 6 Jun 2021 09:02:17 -0500 (ECT)',
 'Sun, 6 Jun 2021 09:01:17 -0500 (ECT)',
 'Sun, 6 Jun 2021 08:57:38 -0500 (ECT)',
 'Fri, 21 May 2021 05:52:58 +0000 (UTC)',
 'Thu, 20 May 2021 20:12:50 +0000 (UTC)',
 'Thu, 03 Jun 2021 17:16:37 +0000 (UTC)',
 'Fri, 7 May 2021 23:51:12 +0000 (UTC)']

In [15]:
#Checking for invalid timestamps in 'Event' column
emails[emails['Event'].isnull()]

,Date,From,Subject,Event


###Transforming ASCII quoted-printable to UTF-8

Encode and Decode MIME <hr>
Many times we need to deal with data which not always has the regular ASCII characters. For example, an email in a different language other than English. Python has mechanism to deal with such characters by using MIME (Multipurpose Internet Mail Extensions) based module.

Quoted-printable in headers <hr>
The character sequence =?UTF-8?Q? is called quoted-printable, and is legitimately used to encode UTF-8 characters in internet headers, since they can contain only ASCII (rfc1342). Quoted-printable is particularly useful for when the content is mostly ASCII, so for example Chris España could be encoded as =?UTF-8?Q?Chris Espa=F1a?=



In [16]:
#Import encode and decode MIME quoted-printable package
import quopri

In [17]:
#Crete function to Transform 'From' and 'Subject' columns from ASCII (quoted-printable) to UTF-8

def parse_quoted_printable(df):
  """Parse from ASCII (quoted-printable) to UTF-8"""
  
  #Split data to lists
  l1_from = df['From'].copy()
  l2_subj = df['Subject'].copy()

  #Create function to decode string
  def decode_quoted(x):
    try:
      return quopri.decodestring(x).decode(encoding='utf-8')
    except UnicodeDecodeError:
      return x

  #Decode each list with quopri
  l1_from = [decode_quoted(i) for i in l1_from]
  l2_subj = [decode_quoted(i) for i in l2_subj]

  #Return dataframe with converted columns to UTF-8
  df = df.assign(From = l1_from,
                 Subject = l2_subj)

  return df

In [18]:
#Apply function to parse from ASCII (quoted-printable) to UTF-8
emails = parse_quoted_printable(emails)

In [ ]:
#Check dataframe after ASCII text transformation
emails.head(10)

In [19]:
#Checking how many values start with text '=?UTF-8?Q?' in 'From' column
my_utf = emails[emails['From'].str.startswith('=?')]['From'].to_list()
my_utf = [i[:10] for i in my_utf]
pd.Series(my_utf).unique()

array(['=?utf-8?Q?', '=?UTF-8?Q?', '=?UTF-8?B?', '=?utf-8?B?',
       '=?iso-8859'], dtype=object)

In [20]:
#Checking how many unique first character are in 'From' column
begins = emails['From'].to_list()
begins = [i[:1] for i in begins]
pd.Series(begins).unique()

array(['=', 'G', '"', 'h', 'Y', 'R', 'A', 'B', 'I', 'n', 'L', 'F', 'b',
       'D', 'M', 'T', 'd', 'E', 'S', '<', 'H', 'C', 'Q', 'J', 'N', 'j',
       's', 'K', 'U', 'g', 'i', 'O', 'z', '', 'X', 'P', 'V', 'v', 'f',
       'W', 'l', 'a'], dtype=object)

In [21]:
#Use regex to replace text starting with =?utf-8?Q? =?UTF-8?Q? =?UTF-8?B? =?utf-8?B? =?iso-8859
emails['From'] = emails['From'].replace(to_replace='=\?\w*-\w[\?|\w]\w[\?|\w]', value='', regex=True)
emails['Subject'] = emails['Subject'].replace(to_replace='=\?\w*-\w[\?|\w]\w[\?|\w]', value='', regex=True)
#emails[emails['From'].str.contains('=\?\w*-\w[\?|\w]\w[\?|\w]', regex=True)]

In [22]:
#Use regex to replace text containing =?=
emails['From'] = emails['From'].replace(to_replace='=*\?=', value='', regex=True)
emails['Subject'] = emails['Subject'].replace(to_replace='=*\?=', value='', regex=True)
#emails[emails['Subject'].str.contains('=*\?=', regex=True)]

In [23]:
emails

,Date,From,Subject,Event
0,6 Jun 2021 11:12:43 -0500,Produbanco_enlínea <bancaenlinea@produbanco.com>,Tm90aWZpY2FjacOzbiBUcmFuc2ZlcmVuY2lhIEVudmlhZG...,2021-06-06 18:12:43+02:00
1,"Sat, 05 Jun 2021 12:57:48 -0700",Google Maps Timeline <noreply-maps-timeline@go...,"🌍_Gabriel,_your_May_update?",2021-06-05 21:57:48+02:00
2,"Mon, 7 Jun 2021 06:55:46 +0000",Elkjøp <elkjop@email.elkjop.no>,RsOlIG1ha3MgdXQgYXYgc29tbWVyZW5zIHNwb3J0c2Jldm...,2021-06-07 08:55:46+02:00
3,"Mon, 07 Jun 2021 14:03:55 -0600","""Comunicados Banco Pichincha"" <comunicados@pic...",Información_de_COSEDE?,2021-06-07 22:03:55+02:00
4,"Sat, 29 May 2021 07:02:53 +0000","""Duolingo"" <hello@duolingo.com>",📝_Your_weekly_progress_report?,2021-05-29 09:02:53+02:00
...,...,...,...,...
1475,"Mon, 9 Dec 2019 09:15:48 +0000",Bob and Julian - PyBites CodeChallenges <newsl...,"PyBites News - PyDev of the Week, Web Scraping...",2019-12-09 10:15:48+01:00
1476,"Fri, 17 Jan 2020 17:47:02 +0100",Aleksander Kolano <alekskol11123@gmail.com>,OD: OD: Hilsen,2020-01-17 17:47:02+01:00
1477,"Sun, 01 Mar 2020 13:02:17 -0500",BANCO PICHINCHA <banco@pichincha.com>,Banca_Electrónica_-_Infor? mación_Código_de_Se...,2020-03-01 19:02:17+01:00
1478,"Wed, 6 Nov 2019 15:29:36 -0500 (ECT)",banco@pichincha.com,TRANSFERENCIA INTERBANCARIA,2019-11-06 21:29:36+01:00


In [16]:
emails['From'][4]

'"Duolingo" <hello@duolingo.com>'

In [24]:
#Checking dataframe info after working with Regex
emails.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1480 entries, 0 to 1479
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype                      
---  ------   --------------  -----                      
 0   Date     1480 non-null   object                     
 1   From     1480 non-null   object                     
 2   Subject  1480 non-null   object                     
 3   Event    1480 non-null   datetime64[ns, Europe/Oslo]
dtypes: datetime64[ns, Europe/Oslo](1), object(3)
memory usage: 46.4+ KB


##Data Manipulation

In [28]:
emails.head()

,Date,From,Subject,Event
0,"Wed, 9 Oct 2019 16:03:38 -0500 (ECT)",banco@pichincha.com,NOTIFICACION BANCO PICHINCHA,2019-10-09 23:03:38+02:00
1,"Thu, 10 Oct 2019 04:55:06 -0500 (ECT)",servicios@titanium.com.ec,-1?Q?Notificaci=F3n_de_Consumos,2019-10-10 11:55:06+02:00
2,13 Oct 2019 13:38:34 -0500,bancavirtual@bancoguayaquil.com,PAGO TARJETAS OTROS BANCOS,2019-10-13 20:38:34+02:00
3,13 Oct 2019 14:29:31 -0500,bancavirtual@bancoguayaquil.com,PAGO TARJETAS OTROS BANCOS,2019-10-13 21:29:31+02:00
4,15 Oct 2019 04:11:07 -0400,"Øyvind_Grimstad,_BearingPoint? <conversations@...",We have received your application!,2019-10-15 10:11:07+02:00


In [ ]:
#Create a function to parse 

emails = emails.assing(Local_part = ,
                       Domain = )


##Data Visualization

In [ ]:
#Import visualization packages
import matplotlib.pyplot as plt
import seaborn as sns